## Data Processing

In [460]:
# Download Data 
#!pip install scipy.io

#import scipy.io
#mat = scipy.io.loadmat('file.mat')

In [461]:

#!pip install sciPy
#!pip install h5py
#import scipy.io
#!pip install tables
#!pip install matlab.engine
#import matlab.engine
#!pip install mat4py

In [462]:
import pickle
import pandas as pd
import numpy as np 
import scipy
import seaborn as sns
import sklearn.metrics 
import h5py
import tables



In [463]:
import scipy.io as sio
#f = sio.loadmat('Data/sub000/After_remarks.mat')
#print(f)
#with open(Data/sub000/A)

In [464]:
#for i in range(0,28):
#    print(f["After_remark"][i][0][0][0],f["After_remark"][i][0][1][0])

In [465]:
#f["After_remark"]

In [466]:
!dir dataset\EEG_Features\DE

 Volume in drive C is Windows-SSD
 Volume Serial Number is C4CE-7D5D

 Directory of c:\Documents\MsC_ITU\2_Semester_MSc_ITU\New_AML\AML_project_gnn\dataset\EEG_Features\DE

04/22/2024  01:11 PM    <DIR>          .
04/22/2024  01:11 PM    <DIR>          ..
04/22/2024  01:11 PM         1,075,366 sub000.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub001.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub002.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub003.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub004.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub005.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub006.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub007.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub008.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub009.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub010.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub011.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub012.pkl.pkl
04/22/2024  01:11 PM         1,075,366 sub

In [467]:
#!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-geometric

In [468]:
#!pip3 install torch torchvision torchaudio
#!pip3 install torchegg

In [469]:
#!pip install torcheeg

In [470]:
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

In [471]:
f = sio.loadmat('dataset/raw_data/sub000/After_remarks.mat')
print(f.keys())
movie_idx_list = []

for i in range(0,28):
    #print(f["After_remark"][i][0][1],f["After_remark"][i][0][2][0][0])#,f["After_remark"][i][0][0])
    movie_idx_list.append(f["After_remark"][i][0][2][0][0])
print(movie_idx_list)

dict_keys(['__header__', '__version__', '__globals__', 'After_remark'])
[11, 8, 2, 5, 6, 9, 3, 12, 20, 17, 26, 23, 13, 15, 14, 16, 10, 1, 4, 7, 28, 25, 22, 19, 18, 21, 27, 24]


# Desciption of array 
- 1) First array is movie 
- 2) Second is Electrodes 
- 3) Third is seconds
- 4) Fourth is frequency bands
- - 4.1: Delta band &rarr; 1-4 Hz, When people are sleeping, stage 2 NREM sleep, woemn seems to have more Delta-activity but is first apparent in early adulthood (30-40)
- - 4.2: Theta band &rarr; 4-8 Hz, learning, memory and spatial Navigation, meditation can increase theta waves, can appear in sleep but not the deepest stages, active stages, the more movement the higher the theta state
- - 4.3: Alpha band &rarr; 8-14 Hz, Two types with Type I being centered around occipital lobe, Type II happens during REM-sleep and is located at frontal-location in the brain, Not significantly linked to people with any major sleep disorder, chronic fatigue syndrome and major depression, alpha waves indicate idlenss, and mistakes are often happening when people are doing something automatically. 
- - 4.4: Beta band  &rarr; 14-30 Hz, can be seperated into three kinds of waves. Associated with waking consciousness, beta waves are accociated with the muscle contractions that happen in isotonic movements indicative of inhibitory cortical transmission mediated by gamma aminobutyric acid (GABA), the principal inhibitory neurotransmitter of the mammalian nervous system. Benzodiazepines, drugs that modulate GABAA receptors, induce beta waves in EEG recordings from humans and rats
- - 4.5: Gamma Band &rarr; 30-47 Hz, Altered gamma activity has been observed in many mood and cognitive disorders such as Alzheimer's disease, epilepsy and schizophrenia For clinical relevance can be biomarker between unipolar and bipolar disorder, decrease Gamma activity is obser in schizophrenia  

# Describtion of metrics: 
- 1) Differential entropy: $DE = \frac{1}{2}ln(2\pi e \sigma^2), Shannon entropy on continuous functions, assumes normal distribution. 
- 2) Power Spectral density: $PSD =E[X^2]$, exists for stationary processes, power of a signal or time series distruted over frequency 
- 3) Where x is the EEG signal and $\sigma$ is the variance of the signal


In [472]:
import torch

In [473]:
f = sio.loadmat('dataset/raw_data/sub000/After_remarks.mat')
clip_seen = f["After_remark"][0][0][2][0][0]

number = f["After_remark"][2][0][2][0][0]

number = torch.from_numpy(np.asarray(number))
#number = Gamma_extract.type(torch.Tensor)
print(number)

tensor(2, dtype=torch.uint8)


In [474]:
# First array is movie 
# Second is Electrodes 
# Third is seconds
# Fourth is frequency bands
import torch
from torcheeg import transforms
from torcheeg.datasets.constants import DEAP_ADJACENCY_MATRIX
from torcheeg.transforms.pyg import ToG
from torcheeg.utils import plot_3d_tensor
import numpy as np


print(DE_feature.shape)

print(PSD_feature.shape)



DE_feature[0][0].shape

Gamma_extract = DE_feature[0,:,:,-1]
#print(Gamma_extract)

#img = plot_3d_tensor(torch.tensor(dataset[0][][0]))

(28, 32, 30, 5)
(28, 32, 30, 5)


In [475]:
#Gamma_extract

In [476]:
### Should we transform???

#t = transforms.ToGrid(Gamma_extract)
#t = transforms.BaselineRemoval()

#eeg = t(eeg=Gamma_extract, baseline=np.random.randn(32, 30))['eeg']
#transformed_eeg = transforms.BandDifferentialEntropy()(eeg=eeg)
#print(transformed_eeg)



In [477]:
import torch
for clip in range(0,28):
    #print(clip)
    Gamma_extract = DE_feature[clip:clip+1,:,:,:]
    Gamma_extract = np.moveaxis(Gamma_extract, 1, 2)

    Gamma_extract = torch.from_numpy(Gamma_extract)
    Gamma_extract = Gamma_extract.type(torch.Tensor)
    number = f["After_remark"][clip][0][2][0][0]
    number = [number]*30
    #print(number)
    target = torch.tensor([number], dtype=torch.long)
    #print(target)
    #print(Gamma_extract[0].shape)

In [478]:
### Helper Fucntion 

def return_binary_val(value):
    if value < 12:
        return 0
    if value> 13 and value <17:
        return 1
    else: 
        return 2

In [479]:
#egg[0][0].shape
for patient in range(0,120):
    #print(str(patient))
    Digits_3 = '{0:03}'.format(patient)
    #print(Digits_3)
    f = sio.loadmat(f'dataset/raw_data/sub{Digits_3}/After_remarks.mat')
    number = f["After_remark"][0][0][2][0][0]
    #print(number)

In [480]:
#DE_feature = pd.read_pickle(r'dataset\EEG_Features\DE\sub000.pkl.pkl')
#PSD_feature = pd.read_pickle(r'dataset\EEG_Features\PSD\sub000.pkl.pkl') 
#print(DE_feature.shape)
for patient in range(0,2):
    Digits_3 = '{0:03}'.format(patient)
    f = sio.loadmat(f'dataset/raw_data/sub{Digits_3}/After_remarks.mat')
    DE_feature = pd.read_pickle(fr'dataset\EEG_Features\DE\sub{Digits_3}.pkl.pkl')
    PSD_feature = pd.read_pickle(fr'dataset\EEG_Features\PSD\sub{Digits_3}.pkl.pkl') 
    print(DE_feature[0][0][0])

[3.37401456 2.20168828 2.28994878 2.70892778 2.7157158 ]
[3.34633713 2.9657934  2.57482702 2.99019405 2.77102368]


In [481]:
#!pip install MNE

In [482]:
from torcheeg.models import DGCNN
import torch.optim as optim
import torch.nn as nn
#f = sio.loadmat('dataset/raw_data/sub000/After_remarks.mat')

#model = DGCNN(in_channels = 5,  num_electrodes=32, hid_channels = 20, num_classes=3)
#optimizer = optim.Adam(model.parameters(), lr=0.01)
#criterion = nn.CrossEntropyLoss()  # Choose an appropriate loss function

# Training loop
#num_epochs = 10


def make_data_set(start_val, end_value):
    data_list = []
    labels_list = []

    for patient in range(start_val, end_value):
        # Load patient data and labels
        digits_3 = '{0:03}'.format(patient)
        f = sio.loadmat(f'dataset/raw_data/sub{digits_3}/After_remarks.mat')
        de_feature = pd.read_pickle(fr'dataset/EEG_Features/DE/sub{digits_3}.pkl.pkl')

        # Temporary storage for this patient's data and labels
        patient_data = []
        patient_labels = []

        for clip in range(28):
            # Prepare input data
            gamma_extract = de_feature[clip:clip + 1, :, :, :]
            gamma_extract = np.moveaxis(gamma_extract, 1, 2)
            inputs = torch.from_numpy(gamma_extract).float()

            # Prepare target data
            number = f["After_remark"][clip][0][2][0][0]
            new_number = return_binary_val(number)
            target = torch.tensor([new_number] * 30, dtype=torch.long)
            
            # Append to patient's data and labels
            patient_data.append(inputs[0])
            patient_labels.append(target)
        
        # Convert patient's data and labels to tensors
        patient_data = torch.stack(patient_data)
        patient_labels = torch.stack(patient_labels)

        # Append this patient's data and labels to the overall lists
        data_list.append(patient_data)
        labels_list.append(patient_labels)

    # Concatenate data and labels along the batch dimension
    data = torch.cat(data_list, dim=0)
    labels = torch.cat(labels_list, dim=0)
    
    return data, labels

X_train, y_train = make_data_set(0,100)
X_val, y_val = make_data_set(100,110)
X_test, y_test = make_data_set(110,120)


In [483]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

torch.Size([2800, 30, 32, 5])
torch.Size([280, 30, 32, 5])
torch.Size([280, 30, 32, 5])


In [484]:
#for i, j in zip(X_train, y_train):
#    print(i.shape,j.shape)

torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 32, 5]) torch.Size([30])
torch.Size([30, 

In [494]:
import torch.nn as nn

# Model definition 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(32, 32, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32,2)
        self.fc1 = nn.Linear(896, 120)
        self.fc2 = nn.Linear(120, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [496]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [486]:
#class Base_CNN(nn.Module):
#   def __init__(self, num_classes=3, n_steps=30, n_features=32):
#       super(Base_CNN, self).__init__()
#       self.model = nn.Sequential(
#           nn.Conv1d(in_channels=n_features, out_channels=64, kernel_size=2),  # Adjust the output channels as needed
#            nn.MaxPool1d(kernel_size=2),
#           nn.Flatten(),
#            nn.Linear(128, 30),  # Calculate the flattened size and adjust as needed
#            nn.ReLU(),
#           nn.Linear(30, num_classes)
#       )#

#    def forward(self, x):
#        return self.model(x)

In [501]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
model.train()  # Set model to training mode
epoch_loss = 0.0
epochs = 100
for data, labels in zip(X_train, y_train):

    
    
    for epoch in range(epochs):
        # Forward pass
        outputs = model(data)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()


        
        # Optimization step
        optimizer.step()
    
            # Calculate average loss for the epoch
        avg_loss = epoch_loss / len(X_train)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")


Validation Loss: 9.7621
Validation F1 Score: 0.3333
Test Loss: 0.9762
Test F1 Score: 0.3333


In [502]:
from sklearn.metrics import f1_score
import torch

# Evaluate the model on validation set
model.eval()  # Set model to evaluation mode

# Validation loop
val_predictions = []
val_labels = []
val_loss = 0.0
with torch.no_grad():
    for data, labels in zip(X_val, y_val):
        outputs = model(data)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        
        # Get predictions
        _, predicted = torch.max(outputs, 1)
        val_predictions.append(predicted)
        val_labels.append(labels)

# Calculate average validation loss
avg_val_loss = val_loss / len(X_val)
print(f"Validation Loss: {avg_val_loss:.4f}")

# Calculate F1 score for validation set
val_f1_score = f1_score(torch.cat(val_labels), torch.cat(val_predictions), average='weighted')
print(f"Validation F1 Score: {val_f1_score:.4f}")

# Evaluate the model on test set
test_predictions = []
test_labels = []
test_loss = 0.0
with torch.no_grad():
    for data, labels in zip(X_test, y_test):
        outputs = model(data)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Get predictions
        _, predicted = torch.max(outputs, 1)
        test_predictions.append(predicted)
        test_labels.append(labels)

# Calculate average test loss
avg_test_loss = test_loss / len(X_test)
print(f"Test Loss: {avg_test_loss:.4f}")

# Calculate F1 score for test set
test_f1_score = f1_score(torch.cat(test_labels), torch.cat(test_predictions), average='weighted')
print(f"Test F1 Score: {test_f1_score:.4f}")


Validation Loss: 0.9762
Validation F1 Score: 0.3333
Test Loss: 0.9762
Test F1 Score: 0.3333


In [ ]:
# test_loss = nn.functional.binary_cross_entropy(output, target).item()
# pred = output
# correct = += int(sum(pred.eq(target)[0]))
# sample_eval ?= int(target.shape[0])
# 